# [Tensorflow - Help Protect the Great Barrier Reef](https://www.kaggle.com/c/tensorflow-great-barrier-reef)
> Wykrywanie Koron cierniowych (rozgwiazd) na rafie koralowej

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/31703/logos/header.png?t=2021-10-29-00-30-04">

# 🐣 Wstęp oraz inne informacje
* Wyniki trzba zgłaszać przy pomocy dostarczonego **python time-series API**
* Każdy wiersz zgłoszenia powinien zawierać wszystkie bbox-y (bounding box-y) na danej klatce filmu. Zgłoszenia powinny być w formacie **COCO** co oznacza: `[x_min, y_min, width, height]`
* Oficjalną metryką zawadów jest: `F2`. Toleruje ona  wyniki false positives(FP), aby zapewnić, że mała ilość rozgwiazd zostanie pominięta. Oznacza to, że eliminacja wyników **false negatives(FN)** jest znacznie ważniejsza niż eliminacja wyników false positives(FP). 
$$F2 = 5 \cdot \frac{precision \cdot recall}{4\cdot precision + recall}$$

* Notatniki, które pomogły wykonać ten notatnik:
1. https://www.kaggle.com/ihorin/great-barrier-reef-yolov5-train-only
2. https://www.kaggle.com/jillanisofttech/great-barrier-reef-yolov5
3. https://www.kaggle.com/awsaf49/great-barrier-reef-yolov5-infer
4. https://www.kaggle.com/awsaf49/great-barrier-reef-yolov5-train

# 🛠 Instalacja bibliotek (wandb i bbox-utility)

In [ ]:
!pip install -qU wandb # check https://github.com/wandb/client for more info
!pip install -qU bbox-utility # check https://github.com/awsaf49/bbox for source code

# 📚 Importowanie bibliotek

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob

from shutil import copyfile
import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')

from joblib import Parallel, delayed

from IPython.display import display

# 📜 WandB
<img src="https://camo.githubusercontent.com/dd842f7b0be57140e68b2ab9cb007992acd131c48284eaf6b1aca758bfea358b/68747470733a2f2f692e696d6775722e636f6d2f52557469567a482e706e67" width=600>

W naszym projekcie wykorzystujemy W&B do śledzenia wyników uczenia oraz samego procesu. Przydaje się również on do zapisywania hiperparametrów i zapisywania wyników eksperymentów. Wszystkie najważniesze funkcje W&B to:

* Przechowywanie hiperparametrów użytych w badaniu treningowym
* Wyszukiwanie, porównywanie i wizualizacja przebiegów treningowych
* Analizuj metryki użycia systemu wraz z przebiegami
* Współpracuj z członkami zespołu
* Replikuj wyniki historyczne
* Wykonuj przeszukiwanie parametrów
* Zachowaj zapisy eksperymentów z dostępem na zawsze


In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    wandb.login(anonymous='must')
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

# 📖 Meta Data

In [ ]:
FOLD      = 1 # which fold to train
DIM       = 4000
MODEL     = 'yolov5s6'
BATCH     = 2
EPOCHS    = 5
OPTMIZER  = 'Adam'

PROJECT   = 'great-barrier-reef-public' # w&b in yolov5
NAME      = f'{MODEL}-dim{DIM}-fold{FOLD}-bat{BATCH}-opt{OPTMIZER}-epch{EPOCHS}' # w&b for yolov5

REMOVE_NOBBOX = True # remove images with no bbox
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
IMAGE_DIR = '/kaggle/images' # directory to save images
LABEL_DIR = '/kaggle/labels' # directory to save labels

## 📁 Stworzenie folderów pod YOLOv5

In [ ]:
!mkdir -p {IMAGE_DIR}
!mkdir -p {LABEL_DIR}

## 📎 Get Paths

In [ ]:
# Train Data
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df['old_image_path'] = f'{ROOT_DIR}/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
df['image_path']  = f'{IMAGE_DIR}/'+df.image_id+'.jpg'
df['label_path']  = f'{LABEL_DIR}/'+df.image_id+'.txt'
df['annotations'] = df['annotations'].progress_apply(eval)
display(df.head(2))

## 📦 Liczba bounding box-ów
* Sprawdzamy ile klatek z filmu posiada bounding box-y

In [ ]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts(normalize=True)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

# 🧹 Czyszczenie zbioru [TEST] trzeba potem przetrenowac model na braku boxów
* uzyjemy tylko **bboxed-images** (`~5k`), z powodu bardzo dużego zbioru (`~23k`), a jednocześnie braku zasobów obliczeniowych do obliczania wag w bardzo dużych modelach (GPU time na kagglu).

In [ ]:
if REMOVE_NOBBOX:
    df = df.query("num_bbox>0")

# ✏️ Zmiana folderu klatek filmu
* Potrzebne są nam wszystkie prawa do obrazka, dlatego zmieniamy ich folder
* możemy ten proces uczynić szybszy przy pomocy **Joblib**, który używa obliczania równoległego.

In [ ]:
def make_copy(row):
    shutil.copyfile(row.old_image_path, row.image_path)
    return

In [ ]:
image_paths = df.old_image_path.tolist()
_ = Parallel(n_jobs=-1, backend='threading')(delayed(make_copy)(row) for _, row in tqdm(df.iterrows(), total=len(df)))

# 🔨 Funkcje pomocnicze (?)

In [ ]:
# check https://github.com/awsaf49/bbox for source code of following utility functions
from bbox.utils import coco2yolo, coco2voc, voc2yolo
from bbox.utils import draw_bboxes, load_image
from bbox.utils import clip_bbox, str2annot, annot2str

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

## Stworzenie kolumny BBox-ów w data frame-ie

In [ ]:
df['bboxes'] = df.annotations.progress_apply(get_bbox)
df.head(2)

## Stowrzenie kolumny wielkości obrazów w data frame-ie
> All Images have same dimension, [Width, Height] =  `[1280, 720]`

In [ ]:
df['width']  = 1280
df['height'] = 720
display(df.head(2))

# 🏷️ Tworzenie etykiet w formie plików `*.txt`
Trzeba przeeksportować etykiety do formatu **YOLO**, z jednym plikiem `*.txt` na każdą klatkę video (jeżeli nie ma bbox-a na klatce, to plik `*.txt` nie jest wymagany). wymagania pliku `*.txt`  wyglądają następująco:

* One row per object
* Each row is class `[x_center, y_center, width, height]` format.
* Box coordinates must be in **normalized** `xywh` format (from `0 - 1`). If your boxes are in pixels, divide `x_center` and `width` by `image width`, and `y_center` and `height` by `image height`.
* Class numbers are **zero-indexed** (start from `0`).

> Competition bbox format is **COCO** hence `[x_min, y_min, width, height]`. So, we need to convert form **COCO** to **YOLO** format.

In [ ]:
cnt = 0
all_bboxes = []
bboxes_info = []
for row_idx in tqdm(range(df.shape[0])):
    row = df.iloc[row_idx]
    image_height = row.height
    image_width  = row.width
    bboxes_coco  = np.array(row.bboxes).astype(np.float32).copy()
    num_bbox     = len(bboxes_coco)
    names        = ['cots']*num_bbox
    labels       = np.array([0]*num_bbox)[..., None].astype(str)
    ## Create Annotation(YOLO)
    with open(row.label_path, 'w') as f:
        if num_bbox<1:
            annot = ''
            f.write(annot)
            cnt+=1
            continue
        bboxes_voc  = coco2voc(bboxes_coco, image_height, image_width)
        bboxes_voc  = clip_bbox(bboxes_voc, image_height, image_width)
        bboxes_yolo = voc2yolo(bboxes_voc, image_height, image_width).astype(str)
        all_bboxes.extend(bboxes_yolo.astype(float))
        bboxes_info.extend([[row.image_id, row.video_id, row.sequence]]*len(bboxes_yolo))
        annots = np.concatenate([labels, bboxes_yolo], axis=1)
        string = annot2str(annots)
        f.write(string)
print('Missing:',cnt)

# 🦾 Data Augmentation 
* Z każdej klatki wideo tworzymy nową klatkę z drobnymi zmianami. zmiemy kontrast, ostrość, saturacje, szum lub dodajemy wszystko naraz (AKA. Fancy Rhea).

In [ ]:
#df = df.reset_index(drop=True)

In [ ]:
import random

def fancy_rhea_augumentation(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    i =  random.randint(0, 4)
    if(i == 0 or i == 4):
        #contrast
        rand = random.uniform(0.3, 1.2)
        alpha = rand
        img = cv2.convertScaleAbs(img, alpha=alpha)
    if (i == 1 or i == 4):
        #sharpness
        kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
        img = cv2.filter2D(img, -1, kernel)
    if (i == 2 or i == 4):
        #saturation
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        (h, s, v) = cv2.split(img_hsv)
        rand = random.choice([2,3,0.5])
        if(rand == 0.5):
            s = s * 2
        else:
            s = s // rand 
        s = np.clip(s,0,255)
        imghsv = cv2.merge([h,s,v])
        img = cv2.cvtColor(imghsv.astype("uint8"), cv2.COLOR_HSV2BGR)
    if (i == 3 or i == 4):
        #noise
        gauss = np.random.normal(0,1,img.size)
        gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
        img = img * gauss + img
    return img

In [ ]:
# for i in range(len(df)):
#     row = df.loc[i]
#     img = fancy_rhea_augumentation(f'{IMAGE_DIR}/{row.image_id}.jpg')
#     cv2.imwrite(f'{IMAGE_DIR}/{row.image_id}_2.jpg', img)
    
#     #Adding to df
#     row.image_path = f'{IMAGE_DIR}/{row.image_id}_2.jpg'
#     df = df.append(row,ignore_index=True)

In [ ]:
# for file_name in os.listdir(LABEL_DIR):
#     if ("_2.txt" in file_name):
#         continue
#     source = LABEL_DIR + "/" + file_name
#     copyfile(source, source.replace(".txt", "_2.txt"))

# 📁 Tworzenie foldów
>  **Cross-Validation**.

In [ ]:
from sklearn.model_selection import GroupKFold
kf = GroupKFold(n_splits = 3)
df = df.reset_index(drop=True)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(df, groups=df.video_id.tolist())):
    df.loc[val_idx, 'fold'] = fold
display(df.fold.value_counts())

# ⭕ `[Data Science]` BBox Distribution

In [ ]:
bbox_df = pd.DataFrame(np.concatenate([bboxes_info, all_bboxes], axis=1),
             columns=['image_id','video_id','sequence',
                     'xmid','ymid','w','h'])
bbox_df[['xmid','ymid','w','h']] = bbox_df[['xmid','ymid','w','h']].astype(float)
bbox_df['area'] = bbox_df.w * bbox_df.h * 1280 * 720
bbox_df = bbox_df.merge(df[['image_id','fold']], on='image_id', how='left')
bbox_df.head(2)

## `x_center` Vs `y_center`

In [ ]:
from scipy.stats import gaussian_kde

all_bboxes = np.array(all_bboxes)

x_val = all_bboxes[...,0]
y_val = all_bboxes[...,1]

# Calculate the point density
xy = np.vstack([x_val,y_val])
z = gaussian_kde(xy)(xy)

fig, ax = plt.subplots(figsize = (12.8, 7.2))
# ax.axis('off')
ax.scatter(x_val, y_val, c=z, s=100, cmap='viridis')
# ax.set_xlabel('x_mid')
# ax.set_ylabel('y_mid')
plt.show()

## `width` Vs `height`

In [ ]:
x_val = all_bboxes[...,2]
y_val = all_bboxes[...,3]

# Calculate the point density
xy = np.vstack([x_val,y_val])
z = gaussian_kde(xy)(xy)

fig, ax = plt.subplots(figsize = (10, 10))
# ax.axis('off')
ax.scatter(x_val, y_val, c=z, s=100, cmap='viridis')
# ax.set_xlabel('bbox_width')
# ax.set_ylabel('bbox_height')
plt.show()

## Area

In [ ]:
import matplotlib as mpl
import seaborn as sns

f, ax = plt.subplots(figsize=(12, 6))
sns.despine(f)

sns.histplot(
    bbox_df,
    x="area", hue="fold",
    multiple="stack",
    palette="viridis",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)
ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter())
ax.set_xticks([500, 1000, 2000, 5000, 10000]);

# 🌈 `[Data Science]` Wizualizacja klatek z BBox-ami

In [ ]:
df2 = df[(df.num_bbox>0)].sample(100) # takes samples with bbox
y = 3; x = 2
plt.figure(figsize=(12.8*x, 7.2*y))
for idx in range(x*y):
    row = df2.iloc[idx]
    img           = load_image(row.image_path)
    image_height  = row.height
    image_width   = row.width
    with open(row.label_path) as f:
        annot = str2annot(f.read())
    bboxes_yolo = annot[...,1:]
    labels      = annot[..., 0].astype(int).tolist()
    names         = ['cots']*len(bboxes_yolo)
    plt.subplot(y, x, idx+1)
    plt.imshow(draw_bboxes(img = img,
                           bboxes = bboxes_yolo, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = 'yolo',
                           line_thickness = 2))
    plt.axis('OFF')
plt.tight_layout()
plt.show()

# 🍚 Dataset do uczenia modelu YOLOv5

In [ ]:
train_files = []
val_files   = []
train_df = df.query("fold!=@FOLD")
valid_df = df.query("fold==@FOLD")
train_files += list(train_df.image_path.unique())
val_files += list(valid_df.image_path.unique())
len(train_files), len(val_files)

# ⚙️ Configuration
Potrzeba stworzyć pliki yaml
1. The dataset root directory path and relative paths to `train / val / test` image directories (or *.txt files with image paths)
2. The number of classes `nc` and 
3. A list of class `names`:`['cots']`

In [ ]:
import yaml

cwd = '/kaggle/working/'

with open(os.path.join( cwd , 'train.txt'), 'w') as f:
    for path in train_df.image_path.tolist():
        f.write(path+'\n')
            
with open(os.path.join(cwd , 'val.txt'), 'w') as f:
    for path in valid_df.image_path.tolist():
        f.write(path+'\n')

data = dict(
    path  = '/kaggle/working',
    train =  os.path.join( cwd , 'train.txt') ,
    val   =  os.path.join( cwd , 'val.txt' ),
    nc    = 1,
    names = ['cots'],
    )

with open(os.path.join( cwd , 'gbr.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join( cwd , 'gbr.yaml'), 'r')
print('\nyaml:')
print(f.read())

In [ ]:
%%writefile /kaggle/working/hyp.yaml
lr0: 0.001  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 4.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.10  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.5  # image mosaic (probability)
mixup: 0.5 # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)

# 📦 [YOLOv5](https://github.com/ultralytics/yolov5/)


In [ ]:
%cd /kaggle/working
!rm -r /kaggle/working/yolov5
!git clone https://github.com/ultralytics/yolov5 # clone
!cp -r /kaggle/input/yolov5-lib-ds /kaggle/working/yolov5
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # check

# 🚅 Training

In [ ]:
!python train.py --img {DIM}\
--batch {BATCH}\
--epochs {EPOCHS}\
--data /kaggle/working/gbr.yaml\
--hyp /kaggle/working/hyp.yaml\
--weights {MODEL}.pt\
--optimizer {OPTMIZER}\
--project {PROJECT} --name {NAME}\
--exist-ok

# ✂️ Usuwanie zbędnych plików (przekazujemy output do INFER)

In [ ]:
!rm -r {IMAGE_DIR}
!rm -r {LABEL_DIR}